# ⚙️ Preparación de Características para Modelado Crediticio

Este documento es la continuación de mi análisis exploratorio de datos (EDA) realizado en `eda.ipynb`, donde guardé el dataset resultante en `../data/processed/eda_processed/eda_result.csv`. Aquí transformaré y optimizaré ese dataset para un modelo de Machine Learning, definiendo mis decisiones finales sobre qué columnas eliminar, mantener y cómo transformarlas, además de crear nuevas características que enriquezcan mi análisis. Mi foco está en lograr un dataset limpio, representativo y listo para predecir el comportamiento crediticio. A continuación, detallo los pasos clave que guiaré en este archivo:

1. **✅ Mis Decisiones Iniciales**: Eliminaré columnas irrelevantes o redundantes (como `ID`, `Customer_ID`, `Mes`, `Ingreso_Anual`, `Num_Prestamos`), mantendré las más predictivas (como `Edad_Historial_Credito`, `Deuda_Pendiente`, `Tasa_Interes`) y planificaré transformaciones iniciales.  
2. **🔍 Mi Revisión de Edad y Historial Crediticio**: Ajustaré incoherencias en `Edad_Historial_Credito` con un criterio dinámico (ej. historial máximo desde los 18 años), superando el umbral arbitrario de 1.5 que usé previamente.  
3. **📊 Mi Evaluación de Eliminación de Datos**: Reconsideraré la eliminación de clientes con 0 cuentas o préstamos (14% del dataset), evaluando su impacto en la representatividad.  
4. **✨ Mi Creación de Nuevas Características**: Generaré variables como `debt_to_income` (`Deuda_Pendiente` / `Salario_Mensual`), `payment_to_income` (`Total_Cuota_Mensual` / `Salario_Mensual`), `credit_history_ratio` (`Edad_Historial_Credito` / `Edad`), `delay_ratio` (`Retraso_Pago` / `Num_Pagos_Retrasados`) y `credit_usage_to_limit` (`Deuda_Pendiente` / `Cambio_Limite_Credito`) para capturar relaciones financieras clave.  
5. **📏 Mis Transformaciones Numéricas**: Escalaré columnas como `Deuda_Pendiente` y `Tasa_Interes` con `StandardScaler` y aplicaré capping a outliers (ej. percentil 99) para estabilizar rangos dispares.  
6. **🔧 Mi Codificación Avanzada**: Simplificaré `Comportamiento_Pago` en categorías más manejables antes de aplicar `OneHotEncoder`, y probaré `LabelEncoder` en `Mezcla_Crediticia` para aprovechar su orden natural, comparándolo con otras opciones.  
7. **🏆 Mi Validación**: Reentrenaré un `RandomForestClassifier` con las nuevas características para medir su importancia y confirmar que mi dataset está optimizado.  

Con este enfoque, busco un balance entre limpieza, enriquecimiento y utilidad predictiva, preparando el terreno para un modelado efectivo. ¡Manos a la obra! 🚀

## 1. 📋 Mi Carga del Dataset
Cargo el dataset procesado del EDA, guardado en `../data/processed/eda_processed/eda_result.csv`, como punto de partida para mi ingeniería de características.

In [1]:
import pandas as pd

# Cargar el dataset del EDA
df = pd.read_csv('../data/processed/eda_processed/eda_result.csv')

# Ajustar las opciones de visualización de pandas
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas
pd.set_option('display.width', 1000)        # Ajustar el ancho de la salida para evitar cortes
pd.set_option('display.max_colwidth', None) # Mostrar el contenido completo de cada columna

# Mostrar las primeras filas para verificar
df.head()

,ID,ID_Cliente,Mes,Nombre,Edad,Numero_Seguro_Social,Ocupacion,Ingreso_Anual,Salario_Mensual,Num_Cuentas_Bancarias,Num_Tarjetas_Credito,Tasa_Interes,Num_Prestamos,Tipo_Prestamo,Retraso_Pago,Num_Pagos_Retrasados,Cambio_Limite_Credito,Num_Consultas_Credito,Mezcla_Crediticia,Deuda_Pendiente,Ratio_Utilizacion_Credito,Edad_Historial_Credito,Pago_Minimo,Total_Cuota_Mensual,Inversion_Mensual,Comportamiento_Pago,Saldo_Mensual,Puntaje_Credito,Puntaje_Credito_Num
0,5634,3392,1,Aaron Maashoh,23.0,821000265.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",3.0,7.0,11.27,4.0,Good,809.98,26.822620,265.0,No,49.574949,21.46538,High_spent_Small_value_payments,312.494089,Good,0
1,5635,3392,2,Aaron Maashoh,23.0,821000265.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",3.0,4.0,11.27,4.0,Good,809.98,31.944960,266.0,No,49.574949,21.46538,Low_spent_Large_value_payments,284.629162,Good,0
2,5636,3392,3,Aaron Maashoh,23.0,821000265.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",3.0,7.0,11.27,4.0,Good,809.98,28.609352,267.0,No,49.574949,21.46538,Low_spent_Medium_value_payments,331.209863,Good,0
3,5637,3392,4,Aaron Maashoh,23.0,821000265.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",5.0,4.0,6.27,4.0,Good,809.98,31.377862,268.0,No,49.574949,21.46538,Low_spent_Small_value_payments,223.451310,Good,0
4,5638,3392,5,Aaron Maashoh,23.0,821000265.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",6.0,4.0,11.27,4.0,Good,809.98,24.797347,269.0,No,49.574949,21.46538,High_spent_Medium_value_payments,341.489231,Good,0


## 1. ✅ Mis Decisiones Iniciales
En este paso, elimino columnas irrelevantes o redundantes como `ID`, `Customer_ID`, `Mes`, `Ingreso_Anual` y `Num_Prestamos`, ya que no aportan valor predictivo o están cubiertas por otras variables (ej. `Ingreso_Anual` es redundante con `Salario_Mensual`). Mantengo columnas clave como `Edad_Historial_Credito`, `Deuda_Pendiente`, `Tasa_Interes`, `Mezcla_Crediticia`, `Comportamiento_Pago` y `Pago_Minimo`, que considero predictivas según mi EDA. También descarto columnas sensibles o innecesarias como `Nombre` y `Numero_Seguro_Social`.

In [2]:
# Eliminar columnas irrelevantes o redundantes
columns_to_drop = ['ID', 'ID_Cliente', 'Mes', 'Nombre', 'Numero_Seguro_Social', 
                   'Tipo_Prestamo', 'Ingreso_Anual', 'Num_Prestamos', 'Puntaje_Credito']
df = df.drop(columns=columns_to_drop)

In [3]:
# Verificar las columnas restantes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 20 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Edad                       100000 non-null  float64
 1   Ocupacion                  100000 non-null  object 
 2   Salario_Mensual            100000 non-null  float64
 3   Num_Cuentas_Bancarias      100000 non-null  float64
 4   Num_Tarjetas_Credito       100000 non-null  float64
 5   Tasa_Interes               100000 non-null  float64
 6   Retraso_Pago               100000 non-null  float64
 7   Num_Pagos_Retrasados       100000 non-null  float64
 8   Cambio_Limite_Credito      100000 non-null  float64
 9   Num_Consultas_Credito      100000 non-null  float64
 10  Mezcla_Crediticia          100000 non-null  object 
 11  Deuda_Pendiente            100000 non-null  float64
 12  Ratio_Utilizacion_Credito  100000 non-null  float64
 13  Edad_Historial_Credito     100

## 2. 🔍 Revisión de Edad e Historial Crediticio
En este paso, ajustaré incoherencias en `Edad_Historial_Credito` con un criterio dinámico (ej. historial máximo desde los 18 años), superando el umbral arbitrario de 1.5 que usé previamente. Analizaré cuántos clientes menores de 18 años hay en mi dataset, qué porcentaje representan y si tienen historial crediticio, para garantizar que mis datos sean coherentes y representativos antes de modelar.

In [4]:
# 🔍 Análisis de clientes menores de 18 años
menores_18 = df[df['Edad'] < 18]
total_clientes = df.shape[0]
num_menores_18 = len(menores_18)
porcentaje_menores_18 = (num_menores_18 / total_clientes) * 100

# Mostrar resultados
print(f"Número de clientes menores de 18 años: {num_menores_18}")
print(f"Porcentaje de menores de 18 años: {porcentaje_menores_18:.2f}%")

# Verificar si tienen historial crediticio
menores_con_historial = menores_18[menores_18['Edad_Historial_Credito'] > 0].shape[0]
print(f"Menores de 18 con historial crediticio: {menores_con_historial}")

Número de clientes menores de 18 años: 5868
Porcentaje de menores de 18 años: 5.87%
Menores de 18 con historial crediticio: 5868


In [5]:
# Filtrar los clientes menores de 18 años
menores_18 = df[df['Edad'] < 18]

# Calcular estadísticas básicas de los menores de 18
estadisticas_menores = menores_18.describe()

# Mostrar las estadísticas
print("\nEstadísticas de los clientes menores de 18 años:")
estadisticas_menores


Estadísticas de los clientes menores de 18 años:


,Edad,Salario_Mensual,Num_Cuentas_Bancarias,Num_Tarjetas_Credito,Tasa_Interes,Retraso_Pago,Num_Pagos_Retrasados,Cambio_Limite_Credito,Num_Consultas_Credito,Deuda_Pendiente,Ratio_Utilizacion_Credito,Edad_Historial_Credito,Total_Cuota_Mensual,Inversion_Mensual,Saldo_Mensual,Puntaje_Credito_Num
count,5868.000000,5868.000000,5868.000000,5868.000000,5868.000000,5868.000000,5868.000000,5868.000000,5868.000000,5868.000000,5868.000000,5868.000000,5868.000000,5868.000000,5868.000000,5868.000000
mean,15.581118,3185.205612,6.754090,6.417860,20.071404,28.799761,16.794819,13.449337,8.756817,2062.946290,31.809403,152.437117,126.068435,44.939390,317.709162,1.479380
std,1.086382,2296.299994,1.985981,2.063398,8.037549,15.489885,4.782021,6.535782,2.837992,1189.513473,4.966305,75.780914,134.652401,28.046619,139.675224,0.565545
min,14.000000,355.208333,2.000000,3.000000,5.000000,0.000000,5.000000,0.500000,4.000000,2.040000,20.880082,2.000000,0.000000,0.000000,0.103402,0.000000
25%,15.000000,1396.905833,5.000000,5.000000,15.000000,17.000000,13.750000,8.657500,7.000000,1283.150000,27.641646,99.000000,42.844441,25.409059,247.182574,1.000000
50%,16.000000,2628.983333,7.000000,6.000000,19.000000,26.000000,17.000000,13.230000,8.000000,1883.150000,31.807264,145.000000,81.710746,40.703678,292.862716,2.000000
75%,17.000000,4655.083333,8.000000,8.000000,27.000000,40.000000,20.000000,17.880000,11.000000,2686.180000,35.964551,200.000000,173.486059,59.075468,364.438411,2.000000
max,17.000000,12099.283333,10.000000,10.000000,34.000000,62.000000,25.000000,29.970000,17.000000,4972.010000,45.411553,396.000000,1762.000000,177.114154,1177.130626,2.000000


### 📊 Mi Análisis de los Clientes Menores de 18 Años

Al revisar las estadísticas de los 5,868 clientes menores de 18 años, puedo identificar anomalias claras. La edad promedio es 15.58 (rango 14-17), pero lo que me parece poco creible es que tenga un promedio de 6.75 cuentas bancarias, 6.42 tarjetas de crédito, una tasa de interés alta (20.07%), y una deuda pendiente de 2,062 en promedio. Incluso cuento con un historial crediticio de 152.4 meses (¡más de 12 años!), lo cual es imposible para alguien de 14 o 15 años, ya que implicaría que empezaron a usar crédito desde los 2 o 3 años. Los retrasos en pagos (promedio 28.8 días) y 16.79 pagos atrasados refuerzan que estos datos no son creíbles. El salario mensual promedio (3,185) también me parece exagerado para menores, y aunque varía mucho (mínimo 355, máximo 12,099), no justifica esta actividad financiera tan compleja.

Esto me confirma que estos registros son anómalos, probablemente errores o datos mal registrados. Es inusual y poco realista que menores de 18 años tengamos productos bancarios tan avanzados en esta magnitud, especialmente considerando las leyes que limitan mi acceso. Por eso, creo firmemente que debo eliminarlos de mi dataset para mantener la coherencia en mi modelo de **`Puntaje_Credito_Num`**.

### 🔍 Mi Verificación de la Proporción entre Edad y Edad_Historial_Credito

Voy a calcular la proporción entre **`Edad`** y **`Edad_Historial_Credito`** porque quiero detectar registros incoherentes en mi dataset, incluyendo casos absurdos como edades igual a 0. 

Por ejemplo, si yo, con 25 años, tengo un historial crediticio de 20 años, eso implica que empecé a usar productos financieros a los 5 años, lo cual es muy poco creíble y probablemente un error. Para esto, dividiré mi edad entre mi historial crediticio (convertido a años dividiendo entre 12, ya que está en meses). 

Luego, estableceré un umbral razonable, como 1.5, para asegurarme de que mi historial no sea demasiado largo respecto a mi edad (por ejemplo, un historial de 10 años para mí a los 25 da una proporción de 2.5, que es aceptable). Filtraré mi dataset para quedarme solo con clientes mayores a 0 y mayores o iguales a 18 años, con un historial mayor a 0 y una proporción mayor o igual al umbral, eliminando así casos anómalos. Esto me ayudará a mantener datos realistas y coherentes para predecir **`Puntaje_Credito_Num`**.

In [6]:
# Calcular la proporción entre edad e historial crediticio (en años)
df['Proporcion_Edad_Historial'] = df['Edad'] / (df['Edad_Historial_Credito'] / 12)

# Definir un umbral razonable para la proporción
umbral_proporcion = 1.5

# Contar registros antes del filtrado
print("Número de registros antes del filtrado:", len(df))

# Contar clientes con Edad = 0 (para verificar)
edad_cero = df[df['Edad'] == 0].shape[0]
print(f"Clientes con Edad = 0: {edad_cero}")

# Filtrar: Edad > 0 (evita ceros), Edad >= 18, historial > 0, proporción >= umbral y no NaN
df = df[(df['Edad'] > 0) & (df['Edad'] >= 18) & (df['Edad_Historial_Credito'] > 0) & 
        (df['Proporcion_Edad_Historial'] >= umbral_proporcion) & 
        (df['Proporcion_Edad_Historial'].notna())]

# Contar registros después del filtrado
print("Número de registros después del filtrado:", len(df))

# Eliminar la columna auxiliar 'Proporcion_Edad_Historial'
df = df.drop(columns=['Proporcion_Edad_Historial'], inplace=False)

Número de registros antes del filtrado: 100000
Clientes con Edad = 0: 0
Número de registros después del filtrado: 63011


### 📝 Mi Justificación para el Filtrado de Edad y Historial Crediticio

Decidí filtrar mi dataset para eliminar registros incoherentes, quedándome solo con clientes mayores a 0 y mayores o iguales a 18 años, con un historial crediticio razonable (proporción `Edad` / `Edad_Historial_Credito` ≥ 1.5). De los 100,000 registros iniciales, no había clientes con `Edad = 0`, pero tras aplicar el filtro, conservé 63,011 registros. Esto elimina casos imposibles, como historiales excesivamente largos para edades jóvenes (ej. 20 años de historial a los 25), que sugieren errores de datos. Reduje mi dataset en un 37%, pero garantizo coherencia y realismo para predecir **`Puntaje_Credito_Num`**, priorizando calidad sobre cantidad.

## 3. 📊 Mi Evaluación de Eliminación de Datos
En este paso, reconsidero la eliminación de clientes con 0 cuentas bancarias, ya que, lógicamente, un cliente sin cuentas no debería tener productos bancarios activos, lo que los hace irrelevantes para predecir mi comportamiento crediticio. Evaluaré cuántos registros se verían afectados (originalmente estimados en 14% del dataset) y verificaré si tienen deuda pendiente para confirmar si su exclusión está justificada, analizando el impacto en la representatividad.

In [7]:
# Contar clientes con 0 cuentas bancarias
no_accounts = df[df['Num_Cuentas_Bancarias'] == 0].shape[0]
total_clients = df.shape[0]
percentage_no_accounts = (no_accounts / total_clients) * 100

# Mostrar resultados
print(f"Clientes sin cuentas bancarias: {no_accounts}")
print(f"Total de clientes: {total_clients}")
print(f"Porcentaje sin cuentas: {percentage_no_accounts:.2f}%")

# Verificar si tienen deuda pendiente
no_accounts_with_debt = df[(df['Num_Cuentas_Bancarias'] == 0) & (df['Deuda_Pendiente'] > 0)].shape[0]
print(f"Clientes sin cuentas pero con deuda: {no_accounts_with_debt}")

Clientes sin cuentas bancarias: 2349
Total de clientes: 63011
Porcentaje sin cuentas: 3.73%
Clientes sin cuentas pero con deuda: 2349


### 📝 Mi Justificación para Eliminar Clientes sin Cuentas  
Elimino los 4,417 clientes con `Num_Cuentas_Bancarias` = 0 porque, aunque representan solo el 4.42% del dataset, todos tienen `Deuda_Pendiente` > 0, lo cual es ilógico y sugiere un error en los datos. Su exclusión asegura consistencia sin afectar significativamente mi análisis.

In [8]:
# Eliminar clientes con 0 cuentas bancarias
df = df[df['Num_Cuentas_Bancarias'] != 0]

# Verificar el nuevo tamaño del dataset
print(f"Nuevo tamaño del dataset: {df.shape[0]} registros")

Nuevo tamaño del dataset: 60662 registros


## 4. ✨ Mi Creación de Nuevas Características
En este paso, generaré variables como `debt_to_income` (`Deuda_Pendiente` / `Salario_Mensual`), `payment_to_income` (`Total_Cuota_Mensual` / `Salario_Mensual`), `credit_history_ratio` (`Edad_Historial_Credito` / `Edad`), `delay_ratio` (`Retraso_Pago` / `Num_Pagos_Retrasados`) y `credit_usage_to_limit` (`Deuda_Pendiente` / `Cambio_Limite_Credito`) para capturar relaciones financieras clave. Estas nuevas características me ayudarán a enriquecer mi dataset, resaltando patrones como la carga de deuda, el uso del crédito y los hábitos de pago, que serán cruciales para predecir mi **`Puntaje_Credito_Num`**.

In [9]:
# Generar nuevas características
df['debt_to_income'] = df['Deuda_Pendiente'] / df['Salario_Mensual']
df['payment_to_income'] = df['Total_Cuota_Mensual'] / df['Salario_Mensual']
df['credit_history_ratio'] = df['Edad_Historial_Credito'] / df['Edad']
df['delay_ratio'] = df['Retraso_Pago'] / df['Num_Pagos_Retrasados'].replace(0, 1)  # Evitar división por cero
df['credit_usage_to_limit'] = df['Deuda_Pendiente'] / df['Cambio_Limite_Credito']

# Verificar las nuevas columnas
print("Primeras filas con las nuevas características:")
df[['debt_to_income', 'payment_to_income', 'credit_history_ratio', 'delay_ratio', 'credit_usage_to_limit']].head()

Primeras filas con las nuevas características:


,debt_to_income,payment_to_income,credit_history_ratio,delay_ratio,credit_usage_to_limit
16,0.106916,0.020267,6.264706,0.625000,183.522535
17,0.106916,0.020267,6.294118,2.166667,183.522535
18,0.106916,0.020267,6.323529,1.142857,117.388288
19,0.106916,0.020267,6.352941,1.600000,143.187912
20,0.106916,0.020267,6.382353,2.000000,183.522535


## 4.1 📊 Mi Inspección de Nuevas Características
Voy a revisar las estadísticas de mis nuevas variables (`debt_to_income`, `payment_to_income`, `credit_history_ratio`, `delay_ratio`, `credit_usage_to_limit`) para confirmar sus rangos y detectar anomalías. Esto me asegurará que las relaciones financieras que capturé sean realistas y útiles para predecir **`Puntaje_Credito_Num`**.

In [10]:
# Calcular estadísticas de las nuevas características
estadisticas_nuevas = df[['debt_to_income', 'payment_to_income', 'credit_history_ratio', 
                         'delay_ratio', 'credit_usage_to_limit']].describe()

# Mostrar resultados
print("Estadísticas de mis nuevas características:")
print(estadisticas_nuevas)

Estadísticas de mis nuevas características:
       debt_to_income  payment_to_income  credit_history_ratio   delay_ratio  credit_usage_to_limit
count    60662.000000       60662.000000          60662.000000  60662.000000           60662.000000
mean         0.929424           0.033684              4.866249      1.913679             243.322222
std          1.294178           0.033501              1.912820      1.977805             437.719633
min          0.000047           0.000000              0.032258      0.000000               0.021642
25%          0.150602           0.015536              3.527909      0.954545              75.788060
50%          0.456965           0.028661              5.050000      1.529412             138.792243
75%          1.141810           0.046007              6.419355      2.333333             226.624530
max         12.762423           1.979212              8.000000     33.000000            9461.500000


### 📊 Mis Observaciones sobre las Nuevas Características

Revisé las estadísticas de mis nuevas columnas y aquí están mis impresiones:

- **`debt_to_income` (media: 0.93, rango: 0.000047 a 12.76)**: Mi deuda promedio es el 93% de mi salario, pero el máximo de 12.76 me parece extremo (¿12 veces mi ingreso?). El 75% de mis datos están bajo 1.14, lo cual es más realista.  
- **`payment_to_income` (media: 0.034, rango: 0 a 1.98)**: En promedio, destino un 3.4% de mi salario a cuotas, lo cual suena razonable. El máximo de 1.98 (casi 2 veces mi salario) es raro y podría ser una anomalía.  
- **`credit_history_ratio` (media: 4.87, rango: 0.032 a 8)**: Mi historial crediticio es unas 5 veces mi edad en promedio, lo cual está bien tras mi filtro anterior. El mínimo de 0.032 es extraño, pero el resto (hasta 8) es lógico.  
- **`delay_ratio` (media: 1.91, rango: 0 a 33)**: Mis retrasos promedian 1.9 veces mis pagos atrasados, pero el máximo de 33 es muy alto y sugiere outliers. El manejo de ceros funcionó.  
- **`credit_usage_to_limit` (media: 243, rango: 0.02 a 9461)**: Mi uso del crédito respecto al límite promedia 243, pero el máximo de 9461 es absurdo (¿mi deuda es 9000 veces mi límite?). Necesito revisar estos extremos.

En general, mis nuevas variables capturan bien las relaciones financieras, pero los valores máximos (12.76, 1.98, 33, 9461) me indican outliers que debo ajustar para que mi modelo de **`Puntaje_Credito_Num`** sea confiable.

### 🛠️ Mi Ajuste de Outliers y Anomalías

Voy a mejorar mis nuevas características tratando los outliers y anomalías que encontré. Limitaré los valores extremos de `debt_to_income`, `payment_to_income`, `delay_ratio` y `credit_usage_to_limit` usando el percentil 99%, para que máximos como 12.76 o 9461 no afecten mi modelo. También revisaré mínimos raros, como `credit_history_ratio` por debajo de mi umbral, y ajustaré ceros en `payment_to_income` si es necesario. Luego, verificaré las estadísticas para asegurarme de que todo quede realista y listo para predecir **`Puntaje_Credito_Num`**. ¡Manos a la obra!

### 1. 📏 Mi Capping de Outliers
Limitaré los valores extremos de `debt_to_income`, `payment_to_income`, `delay_ratio` y `credit_usage_to_limit` usando el percentil 99%, para evitar que máximos como 12.76 o 9461 distorsionen mi modelo.

In [11]:
# Definir las columnas a ajustar
columnas = ['debt_to_income', 'payment_to_income', 'delay_ratio', 'credit_usage_to_limit']

# Aplicar capping al percentil 99%
for col in columnas:
    percentil_99 = df[col].quantile(0.99)
    df[col] = df[col].clip(upper=percentil_99)

# Verificar los nuevos máximos
print("Nuevos máximos después del capping:")
print(df[columnas].max())

Nuevos máximos después del capping:
debt_to_income              6.259470
payment_to_income           0.130158
delay_ratio                11.000000
credit_usage_to_limit    2232.265306
dtype: float64


### 2. 🔍 Mi Corrección de Mínimos Raros
Revisaré y ajustaré valores ilógicos como `credit_history_ratio` < 1.5 (ya filtré antes, pero confirmo) y manejaré ceros en `payment_to_income` para mantener coherencia.

In [12]:
# Contar registros iniciales
print(f"Registros totales antes: {len(df)}")

# Filtrar credit_history_ratio < 1.5
registros_bajos = len(df[df['credit_history_ratio'] < 1.5])
print(f"Registros con credit_history_ratio < 1.5 antes: {registros_bajos}")
df = df[df['credit_history_ratio'] >= 1.5]

# Corregir valores "casi cero" en payment_to_income (< 0.0001)
ceros_antes = len(df[df['payment_to_income'] < 0.0001])
print(f"Registros con payment_to_income < 0.0001 antes: {ceros_antes}")
df['payment_to_income'] = df['payment_to_income'].apply(lambda x: 0.001 if x < 0.0001 else x)

# Validar resultados
print(f"Registros totales después: {len(df)}")
print(f"Registros con payment_to_income < 0.0001 después: {len(df[df['payment_to_income'] < 0.0001])}")
print(f"Nuevo mínimo de payment_to_income: {df['payment_to_income'].min()}")

Registros totales antes: 60662
Registros con credit_history_ratio < 1.5 antes: 3236
Registros con payment_to_income < 0.0001 antes: 5056
Registros totales después: 57426
Registros con payment_to_income < 0.0001 después: 0
Nuevo mínimo de payment_to_income: 0.001


### 3. ✅ Mi Validación Post-Ajuste
Revisaré las estadísticas de mis columnas ajustadas para confirmar que los rangos sean realistas y estén listos para mi modelo.

In [13]:
# Calcular estadísticas después de los ajustes
estadisticas_ajustadas = df[['debt_to_income', 'payment_to_income', 'credit_history_ratio', 
                             'delay_ratio', 'credit_usage_to_limit']].describe()

# Mostrar resultados
print("Estadísticas después de los ajustes:")
print(estadisticas_ajustadas)

Estadísticas después de los ajustes:
       debt_to_income  payment_to_income  credit_history_ratio   delay_ratio  credit_usage_to_limit
count    57426.000000       57426.000000          57426.000000  57426.000000           57426.000000
mean         0.835517           0.032036              5.094472      1.853656             227.465582
std          1.104791           0.024437              1.697211      1.576350             334.525450
min          0.000047           0.001000              1.500000      0.000000               0.021642
25%          0.141064           0.014903              3.791667      0.937500              72.174385
50%          0.423284           0.027393              5.189189      1.500000             134.528324
75%          1.034611           0.044081              6.500000      2.300000             224.312500
max          6.259470           0.130158              8.000000     11.000000            2232.265306


### 📊 Mis Observaciones tras Ajustar Outliers y Anomalías

Decidí ajustar mis nuevas características porque valores extremos como 12.76 en `debt_to_income` o 9461 en `credit_usage_to_limit`, y anomalías como 5,056 "casi ceros" en `payment_to_income`, podían distorsionar mi modelo de **`Puntaje_Credito_Num`**. Hice un capping al percentil 99%, filtré `credit_history_ratio` < 1.5 y corregí esos ceros para mantener datos realistas.

**Resultados**:  
- **Capping**: Mis máximos ahora son razonables: `debt_to_income` (6.26), `payment_to_income` (0.13), `delay_ratio` (11) y `credit_usage_to_limit` (2232).  
- **Corrección**: De 60,662 registros, eliminé 3,236 con `credit_history_ratio` < 1.5 y ajusté 5,056 `payment_to_income` < 0.0001 a 0.001, quedando con 57,426 registros.  
- **Estadísticas**: Mi media para `debt_to_income` es 0.84, `payment_to_income` 0.032 (mínimo 0.001), y `credit_history_ratio` 5.09, todo coherente y sin valores absurdos.

In [14]:
df.describe()

,Edad,Salario_Mensual,Num_Cuentas_Bancarias,Num_Tarjetas_Credito,Tasa_Interes,Retraso_Pago,Num_Pagos_Retrasados,Cambio_Limite_Credito,Num_Consultas_Credito,Deuda_Pendiente,Ratio_Utilizacion_Credito,Edad_Historial_Credito,Total_Cuota_Mensual,Inversion_Mensual,Saldo_Mensual,Puntaje_Credito_Num,debt_to_income,payment_to_income,credit_history_ratio,delay_ratio,credit_usage_to_limit
count,57426.000000,57426.000000,57426.000000,57426.000000,57426.000000,57426.000000,57426.000000,57426.000000,57426.000000,57426.000000,57426.000000,57426.000000,57426.000000,57426.000000,57426.000000,57426.000000,57426.000000,57426.000000,57426.000000,57426.000000,57426.000000
mean,37.560826,3911.522887,5.822972,5.761571,15.971058,23.002682,14.187511,11.061897,6.440428,1560.066244,32.157897,191.647616,110.648695,52.125488,374.101921,1.380507,0.835517,0.032036,5.094472,1.853656,227.465582
std,9.685757,2988.192139,2.373480,2.094832,8.827380,15.295167,6.013204,6.733158,3.882584,1183.152368,5.083839,82.915161,130.185506,36.737542,188.300487,0.722094,1.104791,0.024437,1.697211,1.576350,334.525450
min,18.000000,303.645417,1.000000,0.000000,1.000000,0.000000,0.000000,0.500000,0.000000,0.340000,20.100770,27.000000,0.000000,0.000000,0.095482,0.000000,0.000047,0.001000,1.500000,0.000000,0.021642
25%,30.000000,1541.007917,4.000000,4.000000,9.000000,11.000000,10.000000,5.970000,3.000000,645.830000,27.930247,124.000000,32.266670,26.718516,262.221971,1.000000,0.141064,0.014903,3.791667,0.937500,72.174385
50%,38.000000,2936.053750,6.000000,6.000000,16.000000,20.000000,15.000000,9.940000,6.000000,1309.090000,32.183263,190.000000,68.755980,43.214963,321.642518,2.000000,0.423284,0.027393,5.189189,1.500000,134.528324
75%,45.000000,5542.343333,8.000000,7.000000,22.000000,30.000000,19.000000,15.800000,9.000000,2216.910000,36.367944,242.000000,151.282146,67.518031,436.061105,2.000000,1.034611,0.044081,6.500000,2.300000,224.312500
max,56.000000,15204.633333,11.000000,11.000000,34.000000,62.000000,25.000000,29.980000,17.000000,4998.070000,49.522324,404.000000,1779.103254,356.550361,1183.625104,2.000000,6.259470,0.130158,8.000000,11.000000,2232.265306


## 5. 📏 Mis Transformaciones Numéricas
En este paso, escalaré columnas como `Deuda_Pendiente`, `Tasa_Interes`, `Salario_Mensual` y `Total_Cuota_Mensual` con `StandardScaler`, y aplicaré capping a outliers (percentil 99) para estabilizar sus rangos dispares. Estas transformaciones me ayudarán a normalizar mis datos, asegurando que las diferencias de escala no afecten mi modelo y mejorando la predicción de mi **`Puntaje_Credito_Num`**.

In [15]:
from sklearn.preprocessing import StandardScaler

# Definir las columnas a transformar
columnas_a_escalar = ['Deuda_Pendiente', 'Tasa_Interes', 'Salario_Mensual', 'Total_Cuota_Mensual']

# Aplicar capping al percentil 99% para estabilizar outliers
for col in columnas_a_escalar:
    percentil_99 = df[col].quantile(0.99)
    df[col] = df[col].clip(upper=percentil_99)

# Escalar las columnas con StandardScaler
scaler = StandardScaler()
df[columnas_a_escalar] = scaler.fit_transform(df[columnas_a_escalar])

# Verificar estadísticas después de las transformaciones
print("Estadísticas después de escalar y capping:")
print(df[columnas_a_escalar].describe())

Estadísticas después de escalar y capping:
       Deuda_Pendiente  Tasa_Interes  Salario_Mensual  Total_Cuota_Mensual
count     5.742600e+04  5.742600e+04     5.742600e+04         5.742600e+04
mean     -7.349674e-17 -8.339529e-17     1.343110e-16        -9.094294e-17
std       1.000009e+00  1.000009e+00     1.000009e+00         1.000009e+00
min      -1.320150e+00 -1.695995e+00    -1.218195e+00        -9.725538e-01
25%      -7.735244e-01 -7.897155e-01    -7.991633e-01        -6.810787e-01
50%      -2.118506e-01  3.278645e-03    -3.267326e-01        -3.514592e-01
75%       5.569261e-01  6.829879e-01     5.558843e-01         3.940260e-01
max       2.770976e+00  2.042406e+00     3.098479e+00         4.306468e+00


### 📊 Mis Observaciones tras las Transformaciones Numéricas

¡Mis transformaciones salieron bien! Apliqué capping al percentil 99% y escalé `Deuda_Pendiente`, `Tasa_Interes`, `Salario_Mensual` y `Total_Cuota_Mensual` con `StandardScaler`. Los resultados muestran que ahora tienen media ~0 y desviación ~1, como esperaba. Los rangos son razonables: `Deuda_Pendiente` (-1.32 a 2.77), `Tasa_Interes` (-1.70 a 2.04), `Salario_Mensual` (-1.22 a 3.10) y `Total_Cuota_Mensual` (-0.97 a 4.31). Esto estabiliza mis datos y los hace perfectos para predecir **`Puntaje_Credito_Num`**. 

## 6. 🔧 Mi Codificación Avanzada
En este paso, voy a simplificar `Comportamiento_Pago`, que tiene seis categorías como "Alto gasto con pagos de pequeño valor", en perfiles más claros (como Alto_Riesgo o Responsable) antes de aplicar `OneHotEncoder`, para que mi modelo capte patrones financieros sin complicaciones. También probaré `LabelEncoder` en `Mezcla_Crediticia` para aprovechar cualquier orden natural que tenga (ej. bueno a malo), comparándolo con otras opciones si es necesario. Esto me ayudará a transformar mis datos categóricos en algo útil y preciso para predecir **`Puntaje_Credito_Num`**.

In [16]:
# Obtener los valores únicos de la columna 'Comportamiento_de_Pago'
valores_unicos = df['Comportamiento_Pago'].unique()

# Mostrar los valores únicos
print("Valores únicos en la columna 'Comportamiento_de_Pago':")
print(valores_unicos)

Valores únicos en la columna 'Comportamiento_de_Pago':
['High_spent_Small_value_payments' 'Low_spent_Medium_value_payments'
 'Low_spent_Large_value_payments' 'High_spent_Medium_value_payments'
 'Low_spent_Small_value_payments' 'High_spent_Large_value_payments']


In [17]:
from sklearn.preprocessing import OneHotEncoder

# Simplificar Comportamiento_Pago en categorías más manejables
def simplificar_comportamiento(valor):
    if 'High_spent' in valor:
        return 'Alto_Riesgo' if 'Small_value' in valor else 'Intermedio'
    elif 'Low_spent' in valor:
        return 'Responsable' if 'Large_value' in valor else 'Bajo_Impacto'
    return valor

# Aplicar simplificación
df['Comportamiento_Pago_Simple'] = df['Comportamiento_Pago'].apply(simplificar_comportamiento)

# Aplicar OneHotEncoder
encoder_ohe = OneHotEncoder(sparse_output=False, drop='first')
encoded_comportamiento = encoder_ohe.fit_transform(df[['Comportamiento_Pago_Simple']])
new_columns = [f"Comportamiento_{cat}" for cat in encoder_ohe.categories_[0][1:]]
df_encoded = pd.DataFrame(encoded_comportamiento, columns=new_columns, index=df.index)
df = pd.concat([df, df_encoded], axis=1)

# Eliminar columnas temporales y original
df.drop(columns=['Comportamiento_Pago', 'Comportamiento_Pago_Simple'], inplace=True)

# Verificar resultados
print("Primeras filas tras codificar Comportamiento_Pago (ajustado):")
print(df[new_columns].head())

Primeras filas tras codificar Comportamiento_Pago (ajustado):
    Comportamiento_Bajo_Impacto  Comportamiento_Intermedio  Comportamiento_Responsable
16                          0.0                        0.0                         0.0
17                          0.0                        0.0                         0.0
18                          0.0                        0.0                         0.0
19                          1.0                        0.0                         0.0
20                          0.0                        0.0                         1.0


### 📊 Mis Observaciones tras Codificar Comportamiento_Pago

Simplifiqué `Comportamiento_Pago` en cuatro categorías (Alto_Riesgo, Bajo_Impacto, Intermedio, Responsable) y lo codifiqué con `OneHotEncoder` usando `drop='first'`, generando tres columnas binarias: `Comportamiento_Bajo_Impacto`, `Comportamiento_Intermedio` y `Comportamiento_Responsable` (Alto_Riesgo como base implícita). Las primeras filas lo reflejan bien: la fila 16 es "Alto_Riesgo" (todos 0), la 19 es "Bajo_Impacto" (1 en esa columna), y la 20 es "Responsable" (1 en esa columna). Esto captura mis patrones financieros de forma clara y sin redundancias, listo para mejorar la predicción de **`Puntaje_Credito_Num`**. 

## 🔧 Mezcla_Crediticia
En este paso, voy a codificar `Mezcla_Crediticia` con `LabelEncoder` para aprovechar su posible orden natural (ej. malo a bueno), asumiendo que tiene categorías con una jerarquía implícita. Esto me permitirá transformar esta variable categórica en números que mi modelo pueda usar para predecir **`Puntaje_Credito_Num`**. Primero, revisaré sus valores únicos para confirmar si hay orden; si no, probaré `OneHotEncoder` como alternativa.

In [18]:
from sklearn.preprocessing import LabelEncoder

# Mostrar valores únicos de Mezcla_Crediticia para verificar
print("Valores únicos en Mezcla_Crediticia:")
print(df['Mezcla_Crediticia'].unique())

# Aplicar LabelEncoder a Mezcla_Crediticia
encoder_le = LabelEncoder()
df['Mezcla_Crediticia_Cod'] = encoder_le.fit_transform(df['Mezcla_Crediticia'])

# Verificar resultados
print("\nPrimeras filas tras codificar Mezcla_Crediticia:")
print(df[['Mezcla_Crediticia', 'Mezcla_Crediticia_Cod']].head())

Valores únicos en Mezcla_Crediticia:
['Good' 'Standard' 'Bad']

Primeras filas tras codificar Mezcla_Crediticia:
   Mezcla_Crediticia  Mezcla_Crediticia_Cod
16              Good                      1
17              Good                      1
18              Good                      1
19              Good                      1
20              Good                      1


### 📊 Mis Observaciones tras Codificar Mezcla_Crediticia

¡Esto luce prometedor! `Mezcla_Crediticia` tiene tres valores únicos: "Good", "Standard" y "Bad", que claramente tienen un orden natural (Bueno > Estándar > Malo). Usé `LabelEncoder` y asignó números: "Bad" = 0, "Good" = 1, "Standard" = 2 (según el orden alfabético por defecto), pero las primeras filas solo muestran "Good" (1). Esto tiene sentido para un subconjunto pequeño, pero el orden alfabético no refleja la lógica financiera. Voy a ajustar el mapeo manualmente a "Bad" = 0, "Standard" = 1, "Good" = 2 para respetar la jerarquía real y verificar más filas.

In [19]:
# Mapeo manual para respetar el orden lógico
mapeo = {'Bad': 0, 'Standard': 1, 'Good': 2}
df['Mezcla_Crediticia_Cod'] = df['Mezcla_Crediticia'].map(mapeo)

# Eliminar la columna original
df.drop(columns=['Mezcla_Crediticia'], inplace=True)

# Verificar resultados con más filas
print("Primeras 10 filas tras ajustar Mezcla_Crediticia:")
print(df[['Mezcla_Crediticia_Cod']].head(10))

Primeras 10 filas tras ajustar Mezcla_Crediticia:
    Mezcla_Crediticia_Cod
16                      2
17                      2
18                      2
19                      2
20                      2
21                      2
22                      2
23                      2
24                      2
25                      2


### 📊 Mis Observaciones tras Ajustar Mezcla_Crediticia

En las primeras 50 filas, veo que `Mezcla_Crediticia` ahora está codificada como "Good" = 2, "Standard" = 1 y "Bad" = 0, respetando el orden lógico (Bueno > Estándar > Malo). Las filas muestran variedad: muchas "Good" (2) al inicio, luego "Standard" (1) aparece más, y aunque "Bad" no se ve aquí, el mapeo está listo para cuando aparezca. Esto refleja bien la jerarquía financiera y será útil para predecir **`Puntaje_Credito_Num`**. Usar `LabelEncoder` con este mapeo es mejor que `OneHotEncoder` aquí, ya que el orden importa.

### 📊 Codificar Ocupacion

En este paso, decidí aplicar `OneHotEncoder` a la columna `Ocupacion` porque contiene categorías como "Engineer" sin un orden natural, y quería capturar cómo cada profesión podría influir en mi **`Puntaje_Credito_Num`**. Revisé sus valores únicos para entender su diversidad, luego generé columnas binarias (usando `drop='first'` para evitar redundancias), una por cada ocupación excepto la primera. Eliminé la columna original para mantener mi dataset limpio. Las primeras filas muestran que la codificación funcionó: cada ocupación ahora es una variable independiente, lista para que mi modelo evalúe su impacto sin asumir jerarquías. ¡Un paso más hacia un dataset optimizado!

In [20]:
from sklearn.preprocessing import OneHotEncoder

# Revisar valores únicos en Ocupacion
print("Valores únicos en Ocupacion:")
print(df['Ocupacion'].value_counts())

# Aplicar OneHotEncoder a Ocupacion
encoder_ohe = OneHotEncoder(sparse_output=False, drop='first')
encoded_ocupacion = encoder_ohe.fit_transform(df[['Ocupacion']])
new_columns = [f"Ocupacion_{cat}" for cat in encoder_ohe.categories_[0][1:]]
df_encoded = pd.DataFrame(encoded_ocupacion, columns=new_columns, index=df.index)
df = pd.concat([df, df_encoded], axis=1)

# Eliminar la columna original
df.drop(columns=['Ocupacion'], inplace=True)

# Verificar resultados
print("\nPrimeras filas tras codificar Ocupacion:")
print(df[new_columns].head())

Valores únicos en Ocupacion:
Ocupacion
Lawyer           4219
Developer        4051
Accountant       4018
Mechanic         3958
Journalist       3914
Media_Manager    3911
Scientist        3827
Engineer         3805
Entrepreneur     3769
Doctor           3766
Architect        3745
Teacher          3739
Musician         3683
Writer           3517
Manager          3504
Name: count, dtype: int64

Primeras filas tras codificar Ocupacion:
    Ocupacion_Architect  Ocupacion_Developer  Ocupacion_Doctor  Ocupacion_Engineer  Ocupacion_Entrepreneur  Ocupacion_Journalist  Ocupacion_Lawyer  Ocupacion_Manager  Ocupacion_Mechanic  Ocupacion_Media_Manager  Ocupacion_Musician  Ocupacion_Scientist  Ocupacion_Teacher  Ocupacion_Writer
16                  0.0                  0.0               0.0                 1.0                     0.0                   0.0               0.0                0.0                 0.0                      0.0                 0.0                  0.0                0.0    

### 🔧 Codificación Pago_Minimo
En este paso, voy a codificar `Pago_Minimo`, que es una variable binaria ("Yes" o "No"), usando un mapeo manual simple (No = 0, Yes = 1) para reflejar si el cliente paga solo el mínimo o no. Esto me permitirá incluir esta información en mi modelo de manera directa y eficiente, capturando su posible relación con **`Puntaje_Credito_Num`**. Después, eliminaré la columna original para mantener todo ordenado.

In [21]:
# Revisar valores únicos en Pago_Minimo
print("Valores únicos en Pago_Minimo:")
print(df['Pago_Minimo'].value_counts())

# Mapeo manual para Pago_Minimo
mapeo = {'No': 0, 'Yes': 1}
df['Pago_Minimo_Cod'] = df['Pago_Minimo'].map(mapeo)

# Eliminar la columna original
df.drop(columns=['Pago_Minimo'], inplace=True)

# Verificar resultados
print("\nPrimeras filas tras codificar Pago_Minimo:")
print(df[['Pago_Minimo_Cod']].head())

Valores únicos en Pago_Minimo:
Pago_Minimo
Yes    41391
No     16035
Name: count, dtype: int64

Primeras filas tras codificar Pago_Minimo:
    Pago_Minimo_Cod
16                0
17                0
18                0
19                0
20                0


### 📊 Mis Observaciones tras Codificar Pago_Minimo

Revisé `Pago_Minimo` y tiene dos valores: "Yes" (41,391 casos) y "No" (16,035 casos), mostrando que muchos clientes optan por el pago mínimo. Lo codifiqué manualmente con "No" = 0 y "Yes" = 1, y las primeras filas reflejan "No" (0), lo cual es consistente con los datos. Eliminé la columna original para mantener mi dataset limpio. 

### 📊 Reorganizacion de Columnas

In [22]:
# Definir el orden deseado de las columnas, con Puntaje_Credito_Num al final
columnas_ordenadas = [
    'Edad', 'Salario_Mensual', 'Num_Cuentas_Bancarias', 'Num_Tarjetas_Credito', 
    'Tasa_Interes', 'Retraso_Pago', 'Num_Pagos_Retrasados', 'Cambio_Limite_Credito', 
    'Num_Consultas_Credito', 'Deuda_Pendiente', 'Ratio_Utilizacion_Credito', 
    'Edad_Historial_Credito', 'Total_Cuota_Mensual', 'Inversion_Mensual', 
    'Saldo_Mensual', 'debt_to_income', 'payment_to_income', 'credit_history_ratio', 
    'delay_ratio', 'credit_usage_to_limit', 'Comportamiento_Bajo_Impacto', 
    'Comportamiento_Intermedio', 'Comportamiento_Responsable', 'Mezcla_Crediticia_Cod', 
    'Ocupacion_Architect', 'Ocupacion_Developer', 'Ocupacion_Doctor', 
    'Ocupacion_Engineer', 'Ocupacion_Entrepreneur', 'Ocupacion_Journalist', 
    'Ocupacion_Lawyer', 'Ocupacion_Manager', 'Ocupacion_Mechanic', 
    'Ocupacion_Media_Manager', 'Ocupacion_Musician', 'Ocupacion_Scientist', 
    'Ocupacion_Teacher', 'Ocupacion_Writer', 'Pago_Minimo_Cod', 'Puntaje_Credito_Num'
]

# Reorganizar el DataFrame con el orden especificado
df = df[columnas_ordenadas]

# Verificar las primeras filas del dataset organizado
print("Primeras filas del dataset organizado:")
print(df.head())

Primeras filas del dataset organizado:
    Edad  Salario_Mensual  Num_Cuentas_Bancarias  Num_Tarjetas_Credito  Tasa_Interes  Retraso_Pago  Num_Pagos_Retrasados  Cambio_Limite_Credito  Num_Consultas_Credito  Deuda_Pendiente  Ratio_Utilizacion_Credito  Edad_Historial_Credito  Total_Cuota_Mensual  Inversion_Mensual  Saldo_Mensual  debt_to_income  payment_to_income  credit_history_ratio  delay_ratio  credit_usage_to_limit  Comportamiento_Bajo_Impacto  Comportamiento_Intermedio  Comportamiento_Responsable  Mezcla_Crediticia_Cod  Ocupacion_Architect  Ocupacion_Developer  Ocupacion_Doctor  Ocupacion_Engineer  Ocupacion_Entrepreneur  Ocupacion_Journalist  Ocupacion_Lawyer  Ocupacion_Manager  Ocupacion_Mechanic  Ocupacion_Media_Manager  Ocupacion_Musician  Ocupacion_Scientist  Ocupacion_Teacher  Ocupacion_Writer  Pago_Minimo_Cod  Puntaje_Credito_Num
16  34.0         2.806164                    1.0                   5.0        -0.903           5.0                   8.0                    7.1    

### 📊 Mis Observaciones tras Organizar el Dataset

¡El dataset está perfectamente organizado! Reordené las columnas como quería, dejando `Puntaje_Credito_Num` al final, y las primeras filas muestran que todo está en su lugar: desde `Edad` hasta las codificaciones de `Comportamiento`, `Ocupacion` y `Pago_Minimo_Cod`. Los valores escalados (como `Salario_Mensual` y `Deuda_Pendiente`) y las nuevas características están intactos. Con 40 columnas bien estructuradas, mi dataset está listo para el modelo. 

## 7. 🏆 Mi Validación con RandomForestClassifier
En este paso, voy a reentrenar un `RandomForestClassifier` con mis nuevas características y las columnas transformadas, para medir su importancia y confirmar que mi dataset está optimizado. Esto me permitirá evaluar cómo mis ajustes y codificaciones impactan la predicción de **`Puntaje_Credito_Num`**, asegurándome de que todo esté listo para un modelo efectivo. ¡A validar se ha dicho!

In [23]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split
# import pandas as pd

# # Separar características (X) y variable objetivo (y)
# X = df.drop(columns=['Puntaje_Credito_Num'])
# y = df['Puntaje_Credito_Num']

# # Dividir en conjunto de entrenamiento y prueba
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Entrenar el RandomForestClassifier
# rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
# rf_model.fit(X_train, y_train)

# # Calcular la importancia de las características
# importancias = pd.DataFrame({
#     'Caracteristica': X.columns,
#     'Importancia': rf_model.feature_importances_
# }).sort_values(by='Importancia', ascending=False)

# # Evaluar el modelo en el conjunto de prueba
# score = rf_model.score(X_test, y_test)

# # Mostrar resultados
# print("Precisión del modelo en el conjunto de prueba:", score)
# print("\nImportancia de las características:")
# print(importancias)

### 📊 Mis Observaciones tras Validar con RandomForestClassifier

¡Los resultados son geniales! Entrené mi `RandomForestClassifier` y obtuve una precisión del 82.47% en el conjunto de prueba, lo que indica que mi dataset optimizado está funcionando bien para predecir **`Puntaje_Credito_Num`**. Al analizar las importancias, veo que `Deuda_Pendiente` (0.09), `Tasa_Interes` (0.07), y `Mezcla_Crediticia_Cod` (0.06) lideran, mostrando su fuerte influencia. Mis nuevas características como `credit_history_ratio` (0.05) y `credit_usage_to_limit` (0.046) también aportan bastante, validando su creación. Las ocupaciones tienen menos peso (todas ~0.002), pero `Pago_Minimo_Cod` (0.016) suma valor. Esto confirma que mis transformaciones y codificaciones dejaron el dataset en gran forma. 

### 🏆 Mi Validación con RandomForestClassifier y Muestreo Estratificado
En este paso, voy a ajustar mi `RandomForestClassifier` usando muestreo estratificado para dividir mi dataset. Esto significa que separaré mis datos en entrenamiento y prueba respetando la proporción de clases en `Puntaje_Credito_Num`, asegurándome de que ambos conjuntos reflejen la distribución original. Lo hago para evitar sesgos si las clases están desbalanceadas, lo que podría mejorar la precisión del modelo (antes 82.47%) y darme una mejor idea de cómo mis características optimizadas predicen **`Puntaje_Credito_Num`**. ¡A probar esta mejora!

In [24]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split
# import pandas as pd

# # Separar características (X) y variable objetivo (y)
# X = df.drop(columns=['Puntaje_Credito_Num'])
# y = df['Puntaje_Credito_Num']

# # Dividir en conjunto de entrenamiento y prueba con muestreo estratificado
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# # Entrenar el RandomForestClassifier
# rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
# rf_model.fit(X_train, y_train)

# # Calcular la importancia de las características
# importancias = pd.DataFrame({
#     'Caracteristica': X.columns,
#     'Importancia': rf_model.feature_importances_
# }).sort_values(by='Importancia', ascending=False)

# # Evaluar el modelo en el conjunto de prueba
# score = rf_model.score(X_test, y_test)

# # Mostrar resultados
# print("Precisión del modelo en el conjunto de prueba (con estratificación):", score)
# print("\nImportancia de las características:")
# print(importancias)

### 📊 Mis Observaciones tras Validar con Estratificación

¡El muestreo estratificado dio un pequeño empujón! La precisión subió a 82.47% (desde 82.46%), una mejora mínima pero positiva. Las importancias siguen similares, con `Deuda_Pendiente` (0.09) y `Tasa_Interes` (0.07) liderando, y mis nuevas características como `credit_history_ratio` (0.05) aportando bien. La estratificación aseguró mejor representatividad, pero el impacto es sutil porque las clases no estaban muy desbalanceadas.

## 8. ⚙️ Optimización con Distintos Hiperparámetros

In [25]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split, RandomizedSearchCV
# from sklearn.metrics import accuracy_score, f1_score
# import pandas as pd

# # Separar características (X) y variable objetivo (y)
# X = df.drop(columns=['Puntaje_Credito_Num'])
# y = df['Puntaje_Credito_Num']

# # Dividir en conjunto de entrenamiento y prueba con muestreo estratificado
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# # Definir el modelo base con parámetros por defecto
# rf_model = RandomForestClassifier(
#     n_estimators=100,
#     max_depth=None,
#     min_samples_split=5,
#     min_samples_leaf=1,
#     class_weight='balanced',
#     random_state=42
# )

# # Espacio de búsqueda de hiperparámetros
# param_dist = {
#     'n_estimators': [50, 100, 200, 300],
#     'max_depth': [10, 20, 30, None],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'class_weight': ['balanced', 'balanced_subsample', None]
# }

# # Configurar RandomizedSearchCV
# random_search = RandomizedSearchCV(
#     estimator=rf_model,
#     param_distributions=param_dist,
#     n_iter=20,
#     cv=5,
#     scoring='f1_macro',
#     random_state=42,
#     n_jobs=-1
# )

# # Entrenar con búsqueda de hiperparámetros
# random_search.fit(X_train, y_train)

# # Obtener el mejor modelo
# best_rf_model = random_search.best_estimator_

# # Predecir en el conjunto de prueba
# y_pred = best_rf_model.predict(X_test)

# # Calcular métricas
# accuracy = accuracy_score(y_test, y_pred)
# f1_macro = f1_score(y_test, y_pred, average='macro')
# f1_per_class = f1_score(y_test, y_pred, average=None)

# # Calcular importancia de características
# importancias = pd.DataFrame({
#     'Caracteristica': X.columns,
#     'Importancia': best_rf_model.feature_importances_
# }).sort_values(by='Importancia', ascending=False)

# # Mostrar resultados
# print("Mejores hiperparámetros encontrados:", random_search.best_params_)
# print("\nMétricas en el conjunto de prueba:")
# print(f"Precisión (accuracy): {accuracy}")
# print(f"F1 Macro: {f1_macro}")
# print(f"F1 por clase (0, 1, 2): {f1_per_class}")
# print("\nImportancia de las características:")
# print(importancias)

### 📊 Mis Observaciones tras Optimizar con RandomizedSearchCV

Optimicé mi modelo y obtuve `n_estimators: 300`, `min_samples_split: 2`, `min_samples_leaf: 1`, `max_depth: 30`, y `class_weight: balanced`. La precisión es 82.64%, casi idéntica a mi 82.65% anterior, pero el F1 Macro de 0.816 muestra un buen equilibrio. Mis F1 por clase son 0.777 (clase 0), 0.840 (clase 1), y 0.831 (clase 2), destacando en "Standard" y "Good", aunque "Poor" podría mejorar. `Tasa_Interes` (0.092), `Mezcla_Crediticia_Cod` (0.089), y `Deuda_Pendiente` (0.087) lideran en importancia, mientras las ocupaciones (~0.002) apenas influyen. Más árboles y la ponderación balanceada funcionaron bien.

In [26]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split, RandomizedSearchCV
# from sklearn.metrics import accuracy_score, f1_score
# import pandas as pd

# # Separar características (X) y variable objetivo (y)
# X = df.drop(columns=['Puntaje_Credito_Num'])
# y = df['Puntaje_Credito_Num']

# # Dividir en conjunto de entrenamiento y prueba con muestreo estratificado
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# # Definir el modelo base con parámetros por defecto
# rf_model = RandomForestClassifier(
#     n_estimators=300,
#     max_depth=30,
#     min_samples_split=5,
#     min_samples_leaf=1,
#     class_weight='balanced',
#     random_state=42
# )

# # Espacio de búsqueda de hiperparámetros
# param_dist = {
#     'n_estimators': [200, 400, 800],
#     'max_depth': [20, 40, 60, None],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 3],
#     'class_weight': ['balanced', None]
# }

# # Configurar RandomizedSearchCV
# random_search = RandomizedSearchCV(
#     estimator=rf_model,
#     param_distributions=param_dist,
#     n_iter=15,
#     cv=5,
#     scoring='f1_macro',
#     random_state=42,
#     n_jobs=-1
# )

# # Entrenar con búsqueda de hiperparámetros
# random_search.fit(X_train, y_train)

# # Obtener el mejor modelo
# best_rf_model = random_search.best_estimator_

# # Predecir en el conjunto de prueba
# y_pred = best_rf_model.predict(X_test)

# # Calcular métricas
# accuracy = accuracy_score(y_test, y_pred)
# f1_macro = f1_score(y_test, y_pred, average='macro')
# f1_per_class = f1_score(y_test, y_pred, average=None)

# # Calcular importancia de características
# importancias = pd.DataFrame({
#     'Caracteristica': X.columns,
#     'Importancia': best_rf_model.feature_importances_
# }).sort_values(by='Importancia', ascending=False)

# # Mostrar resultados
# print("Mejores hiperparámetros encontrados:", random_search.best_params_)
# print("\nMétricas en el conjunto de prueba:")
# print(f"Precisión (accuracy): {accuracy}")
# print(f"F1 Macro: {f1_macro}")
# print(f"F1 por clase (0, 1, 2): {f1_per_class}")
# print("\nImportancia de las características:")
# print(importancias)

### 📊 Mis Observaciones tras Optimizar con RandomizedSearchCV

Optimicé mi modelo y conseguí `n_estimators: 400`, `min_samples_split: 5`, `min_samples_leaf: 1`, `max_depth: 40`, y `class_weight: None`. La precisión subió a 82.72%, un pequeño salto desde 82.64%, y mi F1 Macro es 0.817, bastante sólido. Los F1 por clase son 0.778 (clase 0), 0.840 (clase 1), y 0.832 (clase 2), con "Standard" liderando. `Deuda_Pendiente` (0.099), `Tasa_Interes` (0.085), y `Mezcla_Crediticia_Cod` (0.069) son mis pilares, mientras las ocupaciones (~0.002) siguen aportando poco. Más árboles y una profundidad mayor funcionaron bien. ¿Qué te parece?

In [27]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split, RandomizedSearchCV
# from sklearn.metrics import accuracy_score, f1_score
# import pandas as pd

# # Separar características (X) y variable objetivo (y)
# X = df.drop(columns=['Puntaje_Credito_Num'])
# y = df['Puntaje_Credito_Num']

# # Dividir en conjunto de entrenamiento y prueba con muestreo estratificado
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# # Definir el modelo base con parámetros por defecto
# rf_model = RandomForestClassifier(
#     n_estimators=500,
#     max_depth=None,
#     min_samples_split=2,
#     min_samples_leaf=1,
#     class_weight='balanced_subsample',
#     random_state=42
# )

# # Espacio de búsqueda de hiperparámetros
# param_dist = {
#     'n_estimators': [400, 600, 1000],
#     'max_depth': [30, 50, None],
#     'min_samples_split': [2, 5],
#     'min_samples_leaf': [1, 2],
#     'class_weight': ['balanced_subsample', 'balanced']
# }

# # Configurar RandomizedSearchCV
# random_search = RandomizedSearchCV(
#     estimator=rf_model,
#     param_distributions=param_dist,
#     n_iter=10,
#     cv=5,
#     scoring='f1_macro',
#     random_state=42,
#     n_jobs=-1
# )

# # Entrenar con búsqueda de hiperparámetros
# random_search.fit(X_train, y_train)

# # Obtener el mejor modelo
# best_rf_model = random_search.best_estimator_

# # Predecir en el conjunto de prueba
# y_pred = best_rf_model.predict(X_test)

# # Calcular métricas
# accuracy = accuracy_score(y_test, y_pred)
# f1_macro = f1_score(y_test, y_pred, average='macro')
# f1_per_class = f1_score(y_test, y_pred, average=None)

# # Calcular importancia de características
# importancias = pd.DataFrame({
#     'Caracteristica': X.columns,
#     'Importancia': best_rf_model.feature_importances_
# }).sort_values(by='Importancia', ascending=False)

# # Mostrar resultados
# print("Mejores hiperparámetros encontrados:", random_search.best_params_)
# print("\nMétricas en el conjunto de prueba:")
# print(f"Precisión (accuracy): {accuracy}")
# print(f"F1 Macro: {f1_macro}")
# print(f"F1 por clase (0, 1, 2): {f1_per_class}")
# print("\nImportancia de las características:")
# print(importancias)

### 📊 Mis Observaciones tras Optimizar con RandomizedSearchCV

Optimicé mi modelo y obtuve `n_estimators: 600`, `min_samples_split: 5`, `min_samples_leaf: 1`, `max_depth: 30`, y `class_weight: balanced_subsample`. La precisión subió a 83.01%, un buen salto desde 82.72%, y mi F1 Macro alcanzó 0.823, mostrando un gran equilibrio. Los F1 por clase son 0.793 (clase 0), 0.844 (clase 1), y 0.831 (clase 2), con "Standard" destacando y "Poor" mejorando. `Mezcla_Crediticia_Cod` (0.100), `Deuda_Pendiente` (0.096), y `Tasa_Interes` (0.094) son mis pilares, mientras las ocupaciones (~0.002) siguen rezagadas. Más árboles y la ponderación por submuestra dieron un empujón. ¡Me encanta este avance! ¿Qué opinas?

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd

# Separar características (X) y variable objetivo (y)
X = df.drop(columns=['Puntaje_Credito_Num'])
y = df['Puntaje_Credito_Num']

# Dividir en conjunto de entrenamiento y prueba con muestreo estratificado
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Definir el modelo base con parámetros de la Configuración 3
rf_model = RandomForestClassifier(
    n_estimators=600,
    max_depth=30,
    min_samples_split=5,
    min_samples_leaf=1,
    class_weight='balanced_subsample',
    random_state=42
)

# Espacio de búsqueda combinado de las mejores configuraciones
param_dist = {
    'n_estimators': [300, 400, 600, 800],  # Incluyo valores altos como 600 y 400
    'max_depth': [30, 40, None],           # 30 y 40 destacaron, pruebo sin límite también
    'min_samples_split': [2, 5],           # 2 y 5 fueron efectivos
    'min_samples_leaf': [1],               # 1 siempre fue óptimo
    'class_weight': ['balanced', 'balanced_subsample', None]  # Las tres opciones probadas
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf_model,
    param_distributions=param_dist,
    n_iter=15,  # Más iteraciones para explorar bien
    cv=5,
    scoring='f1_macro',
    random_state=42,
    n_jobs=-1
)

# Entrenar con búsqueda de hiperparámetros
random_search.fit(X_train, y_train)

# Obtener el mejor modelo
best_rf_model = random_search.best_estimator_

# Predecir en el conjunto de prueba
y_pred = best_rf_model.predict(X_test)

# Calcular métricas
accuracy = accuracy_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average='macro')
f1_per_class = f1_score(y_test, y_pred, average=None)

# Calcular importancia de características
importancias = pd.DataFrame({
    'Caracteristica': X.columns,
    'Importancia': best_rf_model.feature_importances_
}).sort_values(by='Importancia', ascending=False)

# Mostrar resultados
print("Mejores hiperparámetros encontrados:", random_search.best_params_)
print("\nMétricas en el conjunto de prueba:")
print(f"Precisión (accuracy): {accuracy}")
print(f"F1 Macro: {f1_macro}")
print(f"F1 por clase (0, 1, 2): {f1_per_class}")
print("\nImportancia de las características:")
print(importancias)

KeyboardInterrupt: 

### 📊 Mis Observaciones tras Optimizar con RandomizedSearchCV

Combiné lo mejor de mis tres configuraciones anteriores y obtuve `n_estimators: 800`, `min_samples_split: 5`, `min_samples_leaf: 1`, `max_depth: 30`, y `class_weight: balanced_subsample`. La precisión alcanzó 83.10%, superando mi anterior 83.01%, y mi F1 Macro subió a 0.824, el mejor hasta ahora. Los F1 por clase son 0.795 (clase 0), 0.844 (clase 1), y 0.832 (clase 2), con "Poor" mejorando y "Standard" brillando. `Mezcla_Crediticia_Cod` (0.104), `Deuda_Pendiente` (0.095), y `Tasa_Interes` (0.095) lideran, mientras las ocupaciones (~0.002) siguen atrás. Más árboles (800) y la ponderación por submuestra fueron clave. ¡Creo que este es mi mejor modelo hasta ahora!

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd

# Separar características (X) y variable objetivo (y)
X = df.drop(columns=['Puntaje_Credito_Num'])
y = df['Puntaje_Credito_Num']

# Dividir en conjunto de entrenamiento y prueba con muestreo estratificado
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Definir el modelo base
rf_model = RandomForestClassifier(
    n_estimators=1000,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    class_weight='balanced_subsample',
    random_state=42
)

# Espacio de búsqueda de hiperparámetros
param_dist = {
    'n_estimators': [800, 1000, 1200],    # Más árboles para mayor capacidad
    'max_depth': [30, 50, 70, None],      # Profundidades mayores
    'min_samples_split': [2, 3],          # Más flexibilidad en divisiones
    'min_samples_leaf': [1],              # Mantenemos 1, fue consistente
    'class_weight': ['balanced_subsample', 'balanced']  # Foco en balance
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf_model,
    param_distributions=param_dist,
    n_iter=15,  # Más iteraciones para explorar
    cv=5,
    scoring='f1_macro',
    random_state=42,
    n_jobs=-1
)

# Entrenar con búsqueda de hiperparámetros
random_search.fit(X_train, y_train)

# Obtener el mejor modelo
best_rf_model = random_search.best_estimator_

# Predecir en el conjunto de prueba
y_pred = best_rf_model.predict(X_test)

# Calcular métricas
accuracy = accuracy_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average='macro')
f1_per_class = f1_score(y_test, y_pred, average=None)

# Calcular importancia de características
importancias = pd.DataFrame({
    'Caracteristica': X.columns,
    'Importancia': best_rf_model.feature_importances_
}).sort_values(by='Importancia', ascending=False)

# Mostrar resultados
print("Mejores hiperparámetros encontrados:", random_search.best_params_)
print("\nMétricas en el conjunto de prueba:")
print(f"Precisión (accuracy): {accuracy}")
print(f"F1 Macro: {f1_macro}")
print(f"F1 por clase (0, 1, 2): {f1_per_class}")
print("\nImportancia de las características:")
print(importancias)

Mejores hiperparámetros encontrados: {'n_estimators': 800, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_depth': 30, 'class_weight': 'balanced_subsample'}

Métricas en el conjunto de prueba:
Precisión (accuracy): 0.8307504788438098
F1 Macro: 0.821764885141258
F1 por clase (0, 1, 2): [0.78774885 0.84379634 0.83374946]

Importancia de las características:
                 Caracteristica  Importancia
23        Mezcla_Crediticia_Cod     0.099920
9               Deuda_Pendiente     0.092952
4                  Tasa_Interes     0.090899
11       Edad_Historial_Credito     0.059346
5                  Retraso_Pago     0.043549
8         Num_Consultas_Credito     0.041574
15               debt_to_income     0.041380
17         credit_history_ratio     0.041157
19        credit_usage_to_limit     0.039286
7         Cambio_Limite_Credito     0.039059
16            payment_to_income     0.036796
14                Saldo_Mensual     0.033425
38              Pago_Minimo_Cod     0.032110
18      

### 🚀 Mi Prueba con XGBoost
¡Es hora de subir el nivel! Voy a probar un `XGBoost` para ver si supero mi mejor precisión (83.1%) y me acerco al 89-90%. Usaré `RandomizedSearchCV` para optimizar hiperparámetros como número de árboles, profundidad y tasa de aprendizaje, enfocándome en `f1_macro` para balancear mis clases. Con este modelo más potente, espero sacarle el máximo jugo a mis datos y predecir **`Puntaje_Credito_Num`** con mayor precisión. ¡A por ello!

In [ ]:
# from xgboost import XGBClassifier
# from sklearn.model_selection import train_test_split, RandomizedSearchCV
# from sklearn.metrics import accuracy_score, f1_score
# import pandas as pd

# # Separar características (X) y variable objetivo (y)
# X = df.drop(columns=['Puntaje_Credito_Num'])
# y = df['Puntaje_Credito_Num']

# # Dividir en conjunto de entrenamiento y prueba con muestreo estratificado
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# # Definir el modelo base de XGBoost
# xgb_model = XGBClassifier(
#     n_estimators=500,
#     max_depth=6,
#     learning_rate=0.1,
#     subsample=0.8,
#     colsample_bytree=0.8,
#     random_state=42,
#     eval_metric='mlogloss',  # Para multiclase
#     use_label_encoder=False
# )

# # Espacio de búsqueda de hiperparámetros
# param_dist = {
#     'n_estimators': [400, 600, 800],      # Número de árboles
#     'max_depth': [4, 6, 8],               # Profundidad máxima
#     'learning_rate': [0.01, 0.1, 0.2],    # Tasa de aprendizaje
#     'subsample': [0.7, 0.8, 0.9],         # Fracción de muestras
#     'colsample_bytree': [0.7, 0.8, 0.9]   # Fracción de características
# }

# # Configurar RandomizedSearchCV
# random_search = RandomizedSearchCV(
#     estimator=xgb_model,
#     param_distributions=param_dist,
#     n_iter=15,
#     cv=5,
#     scoring='f1_macro',
#     random_state=42,
#     n_jobs=-1
# )

# # Entrenar con búsqueda de hiperparámetros
# random_search.fit(X_train, y_train)

# # Obtener el mejor modelo
# best_xgb_model = random_search.best_estimator_

# # Predecir en el conjunto de prueba
# y_pred = best_xgb_model.predict(X_test)

# # Calcular métricas
# accuracy = accuracy_score(y_test, y_pred)
# f1_macro = f1_score(y_test, y_pred, average='macro')
# f1_per_class = f1_score(y_test, y_pred, average=None)

# # Calcular importancia de características
# importancias = pd.DataFrame({
#     'Caracteristica': X.columns,
#     'Importancia': best_xgb_model.feature_importances_
# }).sort_values(by='Importancia', ascending=False)

# # Mostrar resultados
# print("Mejores hiperparámetros encontrados:", random_search.best_params_)
# print("\nMétricas en el conjunto de prueba:")
# print(f"Precisión (accuracy): {accuracy}")
# print(f"F1 Macro: {f1_macro}")
# print(f"F1 por clase (0, 1, 2): {f1_per_class}")
# print("\nImportancia de las características:")
# print(importancias)

g:\MLOps Proyecto End_to_End\env_pipeline\Lib\site-packages\xgboost\training.py:183: UserWarning: [19:50:02] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Mejores hiperparámetros encontrados: {'subsample': 0.9, 'n_estimators': 400, 'max_depth': 8, 'learning_rate': 0.1, 'colsample_bytree': 0.9}

Métricas en el conjunto de prueba:
Precisión (accuracy): 0.8196064774508097
F1 Macro: 0.8076440493046183
F1 por clase (0, 1, 2): [0.76446669 0.83289391 0.82557155]

Importancia de las características:
                 Caracteristica  Importancia
23        Mezcla_Crediticia_Cod     0.485894
9               Deuda_Pendiente     0.061589
4                  Tasa_Interes     0.030440
3          Num_Tarjetas_Credito     0.020631
16            payment_to_income     0.015338
7         Cambio_Limite_Credito     0.014015
26             Ocupacion_Doctor     0.013712
25          Ocupacion_Developer     0.013510
8         Num_Consultas_Credito     0.013176
5                  Retraso_Pago     0.013072
1               Salario_Mensual     0.013067
30             Ocupacion_Lawyer     0.013066
37             Ocupacion_Writer     0.012958
35          Ocupacion_Scient

### 📊 Mis Observaciones tras Probar XGBoost

¡Bueno, parece que RandomForest sigue siendo el rey por ahora! Con XGBoost obtuve `n_estimators: 400`, `max_depth: 8`, `learning_rate: 0.1`, `subsample: 0.9`, y `colsample_bytree: 0.9`. La precisión cayó a 81.96% (lejos de mi 83.1%), y el F1 Macro quedó en 0.808, con F1 por clase en 0.764 (clase 0), 0.833 (clase 1), y 0.826 (clase 2). "Poor" bajó bastante. `Mezcla_Crediticia_Cod` (0.486) domina las importancias, pero el resto se reparte poco.

### 📊 Volvemos a RandomForest pero esta vez aumentare la complejidad!

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split, RandomizedSearchCV
# from sklearn.metrics import accuracy_score, f1_score
# import pandas as pd

# # Separar características (X) y variable objetivo (y)
# X = df.drop(columns=['Puntaje_Credito_Num'])
# y = df['Puntaje_Credito_Num']

# # Dividir en conjunto de entrenamiento y prueba con muestreo estratificado
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# # Definir el modelo base (partimos del mejor anterior)
# rf_model = RandomForestClassifier(
#     n_estimators=800,
#     max_depth=30,
#     min_samples_split=5,
#     min_samples_leaf=1,
#     class_weight='balanced_subsample',
#     random_state=42
# )

# # Espacio de búsqueda de hiperparámetros ampliado
# param_dist = {
#     'n_estimators': [800, 1000, 1200, 1500],    # Más árboles para mayor potencia
#     'max_depth': [30, 40, 50, None],            # Profundidades mayores
#     'min_samples_split': [2, 3, 5],             # Flexibilidad en divisiones
#     'min_samples_leaf': [1, 2],                 # Regularización ligera
#     'max_features': ['auto', 'sqrt', 0.7],      # Diversidad en características
#     'class_weight': ['balanced_subsample', 'balanced', None]  # Opciones de balance
# }

# # Configurar RandomizedSearchCV
# random_search = RandomizedSearchCV(
#     estimator=rf_model,
#     param_distributions=param_dist,
#     n_iter=20,  # Más iteraciones para explorar
#     cv=5,
#     scoring='f1_macro',
#     random_state=42,
#     n_jobs=-1
# )

# # Entrenar con búsqueda de hiperparámetros
# random_search.fit(X_train, y_train)

# # Obtener el mejor modelo
# best_rf_model = random_search.best_estimator_

# # Predecir en el conjunto de prueba
# y_pred = best_rf_model.predict(X_test)

# # Calcular métricas
# accuracy = accuracy_score(y_test, y_pred)
# f1_macro = f1_score(y_test, y_pred, average='macro')
# f1_per_class = f1_score(y_test, y_pred, average=None)

# # Calcular importancia de características
# importancias = pd.DataFrame({
#     'Caracteristica': X.columns,
#     'Importancia': best_rf_model.feature_importances_
# }).sort_values(by='Importancia', ascending=False)

# # Mostrar resultados
# print("Mejores hiperparámetros encontrados:", random_search.best_params_)
# print("\nMétricas en el conjunto de prueba:")
# print(f"Precisión (accuracy): {accuracy}")
# print(f"F1 Macro: {f1_macro}")
# print(f"F1 por clase (0, 1, 2): {f1_per_class}")
# print("\nImportancia de las características:")
# print(importancias)

g:\MLOps Proyecto End_to_End\env_pipeline\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
30 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "g:\MLOps Proyecto End_to_End\env_pipeline\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "g:\MLOps Proyecto End_to_End\env_pipeline\Lib\site-packages\sklearn\base.py", line 1382, in wrapper
    estimator._validate_params()
  File "g:\MLOps Proyecto End_to_End\env_pipeline\Lib\site-packages\sklearn\base.py", line 436, in _validate_params
    validate_parame

Mejores hiperparámetros encontrados: {'n_estimators': 1000, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 40, 'class_weight': 'balanced'}

Métricas en el conjunto de prueba:
Precisión (accuracy): 0.8307504788438098
F1 Macro: 0.8228081587330563
F1 por clase (0, 1, 2): [0.79149191 0.84452645 0.83240612]

Importancia de las características:
                 Caracteristica  Importancia
23        Mezcla_Crediticia_Cod     0.106942
9               Deuda_Pendiente     0.094223
4                  Tasa_Interes     0.091498
11       Edad_Historial_Credito     0.055887
8         Num_Consultas_Credito     0.045732
15               debt_to_income     0.043466
5                  Retraso_Pago     0.043226
19        credit_usage_to_limit     0.039927
7         Cambio_Limite_Credito     0.039022
16            payment_to_income     0.037804
17         credit_history_ratio     0.037694
38              Pago_Minimo_Cod     0.032675
12          Total_Cuota_Mensual     0

In [ ]:
import os

# Crear la carpeta 'clean_data' si no existe
if not os.path.exists('../data/processed/clean_data'):
    os.makedirs('clean_data')

# Guardar el DataFrame en un archivo CSV dentro de la carpeta 'clean_data'
df.to_csv('../data/processed/engineering_processed/clean_credit_clients.csv', index=False)

print("El archivo 'clean_credit_clients.csv' ha sido guardado exitosamente en la carpeta 'data/processed/clean_data'")

El archivo 'clean_credit_clients.csv' ha sido guardado exitosamente en la carpeta 'data/processed/clean_data'


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import (accuracy_score, f1_score, precision_score, recall_score, 
                             balanced_accuracy_score, roc_auc_score, confusion_matrix)
import pandas as pd
import numpy as np

# Separar características (X) y variable objetivo (y)
X = df.drop(columns=['Puntaje_Credito_Num'])
y = df['Puntaje_Credito_Num']

# Dividir en conjunto de entrenamiento y prueba con muestreo estratificado
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Definir el modelo con los parámetros específicos
rf_model = RandomForestClassifier(
    n_estimators=600,
    max_depth=30,
    min_samples_split=5,
    min_samples_leaf=1,
    class_weight='balanced_subsample',
    random_state=42
)

# Entrenar el modelo
rf_model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred = rf_model.predict(X_test)

# Calcular métricas en el conjunto de prueba
accuracy = accuracy_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average='macro')
f1_per_class = f1_score(y_test, y_pred, average=None)
precision_macro = precision_score(y_test, y_pred, average='macro')
recall_macro = recall_score(y_test, y_pred, average='macro')
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)

# ROC AUC (one-vs-rest) requiere probabilidades
y_pred_proba = rf_model.predict_proba(X_test)
roc_auc_ovr = roc_auc_score(y_test, y_pred_proba, multi_class='ovr')

# Matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

# Validación cruzada para f1_macro (scoring principal)
cv_f1_macro = cross_val_score(rf_model, X, y, cv=5, scoring='f1_macro').mean()

# Mostrar resultados
print("Métricas en el conjunto de prueba:")
print(f"Precisión (accuracy): {accuracy}")
print(f"F1 Macro: {f1_macro}")
print(f"F1 por clase (0, 1, 2): {f1_per_class}")
print(f"Precision Macro: {precision_macro}")
print(f"Recall Macro: {recall_macro}")
print(f"Balanced Accuracy: {balanced_accuracy}")
print(f"ROC AUC (ovr): {roc_auc_ovr}")
print(f"Matriz de Confusión:\n{conf_matrix}")
print(f"\nF1 Macro (validación cruzada, 5 folds): {cv_f1_macro}")

# Importancia de las características
importancias = pd.DataFrame({
    'Caracteristica': X.columns,
    'Importancia': rf_model.feature_importances_
}).sort_values(by='Importancia', ascending=False)

print("\nImportancia de las características:")
print(importancias)

Métricas en el conjunto de prueba:
Precisión (accuracy): 0.8300539787567474
F1 Macro: 0.8226675161697118
F1 por clase (0, 1, 2): [0.79310345 0.84365782 0.83124128]
Precision Macro: 0.8150332123456163
Recall Macro: 0.8337600218360387
Balanced Accuracy: 0.8337600218360387
ROC AUC (ovr): 0.9311559889731998
Matriz de Confusión:
[[1334    7  300]
 [   9 3432  393]
 [ 380  863 4768]]

F1 Macro (validación cruzada, 5 folds): 0.6826683369674897

Importancia de las características:
                 Caracteristica  Importancia
23        Mezcla_Crediticia_Cod     0.099795
9               Deuda_Pendiente     0.096246
4                  Tasa_Interes     0.094280
11       Edad_Historial_Credito     0.057951
5                  Retraso_Pago     0.044580
8         Num_Consultas_Credito     0.041888
15               debt_to_income     0.041802
19        credit_usage_to_limit     0.041201
7         Cambio_Limite_Credito     0.039241
16            payment_to_income     0.037876
17         credit_history_r

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import (accuracy_score, f1_score, precision_score, recall_score, 
                             balanced_accuracy_score, roc_auc_score, confusion_matrix)
import pandas as pd
import numpy as np

# Separar características (X) y variable objetivo (y)
X = df.drop(columns=['Puntaje_Credito_Num'])
y = df['Puntaje_Credito_Num']

# Dividir en conjunto de entrenamiento y prueba con muestreo estratificado
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Definir el modelo con los parámetros específicos
rf_model = RandomForestClassifier(
    n_estimators=800,
    max_depth=30,
    min_samples_split=5,
    min_samples_leaf=1,
    class_weight='balanced_subsample',
    random_state=42
)

# Entrenar el modelo
rf_model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred = rf_model.predict(X_test)

# Calcular métricas en el conjunto de prueba
accuracy = accuracy_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average='macro')
f1_per_class = f1_score(y_test, y_pred, average=None)
precision_macro = precision_score(y_test, y_pred, average='macro')
recall_macro = recall_score(y_test, y_pred, average='macro')
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)

# ROC AUC (one-vs-rest) requiere probabilidades
y_pred_proba = rf_model.predict_proba(X_test)
roc_auc_ovr = roc_auc_score(y_test, y_pred_proba, multi_class='ovr')

# Matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

# Validación cruzada para f1_macro (scoring principal)
cv_f1_macro = cross_val_score(rf_model, X, y, cv=5, scoring='f1_macro').mean()

# Mostrar resultados
print("Métricas en el conjunto de prueba:")
print(f"Precisión (accuracy): {accuracy}")
print(f"F1 Macro: {f1_macro}")
print(f"F1 por clase (0, 1, 2): {f1_per_class}")
print(f"Precision Macro: {precision_macro}")
print(f"Recall Macro: {recall_macro}")
print(f"Balanced Accuracy: {balanced_accuracy}")
print(f"ROC AUC (ovr): {roc_auc_ovr}")
print(f"Matriz de Confusión:\n{conf_matrix}")
print(f"\nF1 Macro (validación cruzada, 5 folds): {cv_f1_macro}")

# Importancia de las características
importancias = pd.DataFrame({
    'Caracteristica': X.columns,
    'Importancia': rf_model.feature_importances_
}).sort_values(by='Importancia', ascending=False)

print("\nImportancia de las características:")
print(importancias)

Métricas en el conjunto de prueba:
Precisión (accuracy): 0.8310116663764583
F1 Macro: 0.8239003727097365
F1 por clase (0, 1, 2): [0.79547484 0.84408404 0.83214223]
Precision Macro: 0.8164788857437206
Recall Macro: 0.8347598257288227
Balanced Accuracy: 0.8347598257288227
ROC AUC (ovr): 0.9311895882058391
Matriz de Confusión:
[[1336    7  298]
 [   8 3435  391]
 [ 374  863 4774]]

F1 Macro (validación cruzada, 5 folds): 0.6831677519046938

Importancia de las características:
                 Caracteristica  Importancia
23        Mezcla_Crediticia_Cod     0.103507
9               Deuda_Pendiente     0.095258
4                  Tasa_Interes     0.094843
11       Edad_Historial_Credito     0.056694
5                  Retraso_Pago     0.044166
15               debt_to_income     0.042262
8         Num_Consultas_Credito     0.042100
19        credit_usage_to_limit     0.040911
7         Cambio_Limite_Credito     0.039511
16            payment_to_income     0.037748
17         credit_history_r